Read pose data from vicon csv file in and save in `radar_odometry.py` compatible format.
Basically, it reads in quaternion \[x,y,x,p,q,r,w\] format and converts it to \[1x12\] transformation matrix format.

In [1]:
import sys
sys.path.append('..')

In [2]:
import numpy as np
import conversions as tgm
import torch

In [3]:
%env CUDA_VISIBLE_DEVICES=4

env: CUDA_VISIBLE_DEVICES=4


In [4]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [5]:
print(device)

cuda


In [6]:
f_vicon = '/mnt/datasets/yasin/records2-23mart2020/vicon-data/2020-03-23-18-42-55-test1.csv'
#f_vicon = '/mnt/datasets/yasin/records2-23mart2020/vicon-data/2020-03-23-18-50-27-test2.csv'
vicon_full = np.genfromtxt(f_vicon, delimiter=',', skip_header=True)
print(vicon_full.shape)

(7928, 12)


In [7]:
# xyzpqrw
vicon = vicon_full[:,5:]
print(vicon.shape)

(7928, 7)


In [8]:
# Number of radar frames in seqs
n_test = 1993 #test1
#n_test = 2047+1078 #test2

In [9]:
# Generate random uniformly distributed n_test1 indices
rng = np.random.default_rng()
ind_perm = rng.permutation(vicon.shape[0])
ind_sample = sorted(ind_perm[:n_test])
# print(ind_sample)

In [10]:
vicon_sample = vicon[ind_sample] #pqrw
vicon_sample[:,[3,6]] = vicon_sample[:,[6,3]] #wpqr
print(vicon_sample.shape)

(1993, 7)


In [63]:
vs_eul_r = tgm.quaternion_to_angle_axis(torch.Tensor(vicon_sample[:,3:]).to(device))
print(vs_eul_r.shape)

torch.Size([1993, 3])


In [64]:
vs_eul_t = torch.Tensor(vicon_sample[:,:3]).to(device)
print(vs_eul_t.shape)

torch.Size([1993, 3])


In [65]:
vs_eul = torch.cat([vs_eul_t, vs_eul_r], dim=1)
print(vs_eul.shape)

torch.Size([1993, 6])


In [66]:
vs_tform = tgm.rtvec_to_pose(vs_eul)
print(vs_tform.shape)

torch.Size([1993, 4, 4])


In [67]:
vs_tform = vs_tform[:,:3,:]
print(vs_tform.shape)

torch.Size([1993, 3, 4])


In [68]:
vs_tform_flat = vs_tform.flatten(start_dim=1)
print(vs_tform_flat.shape)

torch.Size([1993, 12])


In [69]:
np_vs_tform_flat = vs_tform_flat.cpu().numpy()
print(np_vs_tform_flat.shape)

(1993, 12)


In [70]:
np.savetxt('vicon-gt-test1.csv', np_vs_tform_flat, delimiter=',')

In [11]:
# Save xyz only
np.savetxt('vicon-gt-test1-xyz.csv', vicon_sample[:,:3], delimiter=',')

# Check quaternion conversion

In [72]:
vs_q_r = tgm.angle_axis_to_quaternion(vs_eul_r)
print(vs_q_r.shape)

torch.Size([1993, 4])


In [73]:
# the both must be equal
print(vs_q_r[0])
print(vicon_sample[0])

tensor([ 0.9867,  0.0271,  0.1587, -0.0237], device='cuda:0')
[-1.29174713  3.06130484  0.78150638  0.98666533  0.02710472  0.15872805
 -0.02371204]
